In [109]:
import pandas as pd
import nltk
import emoji
import re
from IPython.display import display

pd.set_option('display.max_columns', None) # or 1000.

In [147]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [55]:
df1 = pd.read_json('formulae-snscrape.json', lines=True)
df2 = pd.read_csv('formula_e.csv')
print(df1.info())
print(df2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   _type             10000 non-null  object             
 1   url               10000 non-null  object             
 2   date              10000 non-null  datetime64[ns, UTC]
 3   content           10000 non-null  object             
 4   renderedContent   10000 non-null  object             
 5   id                10000 non-null  int64              
 6   user              10000 non-null  object             
 7   replyCount        10000 non-null  int64              
 8   retweetCount      10000 non-null  int64              
 9   likeCount         10000 non-null  int64              
 10  quoteCount        10000 non-null  int64              
 11  conversationId    10000 non-null  int64              
 12  lang              10000 non-null  object             
 13  so

In [61]:
df2 = df2.drop('Unnamed: 0', axis=1)
df2.columns

Index(['id', 'conversation_id', 'created_at', 'date', 'time', 'timezone',
       'user_id', 'username', 'name', 'place', 'tweet', 'language', 'mentions',
       'urls', 'photos', 'replies_count', 'retweets_count', 'likes_count',
       'hashtags', 'cashtags', 'link', 'retweet', 'quote_url', 'video',
       'thumbnail', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest'],
      dtype='object')

In [86]:
df2_indo = df2[df2.language == 'in']
df2_indo.tweet

0       Gembong-PDIP Curiga Anies Paksa ASN DKI Beli T...
1       KPK tegaskan tak main-main usut dugaan korupsi...
2       Pengaspalan sirkuit Formula E Jakarta di kawas...
3       Dia, melabali pengkritik sebagai pembebci...!!...
4       KPK Harus Naikkan Status Kasus Formula E ke Pe...
                              ...                        
7159    Wakil Ketua Komisi E DPRD DKI Anggara Wicitra ...
7160    @AlfathMauzy @f1speed_indo "Wow ini lah tampak...
7162      @asumsico Kirain formula e yg mau ditambahin jg
7163    @danugraha54 @YZhireng @haluanghabang @Dennysi...
7166    Kapan Tiket Formula E Mulai Dijual? Begini Jaw...
Name: tweet, Length: 2960, dtype: object

In [70]:
df1_indo = df1[df1.lang == 'in']
df1_indo['content']

0       Dalam dua bulan Jakarta akan menjadi tuan ruma...
1       KPK tegaskan tak main-main usut dugaan korupsi...
2       Pengaspalan sirkuit Formula E Jakarta di kawas...
3       https://t.co/CJ0pw6Hu0O\nKlo dilaporin dalangn...
4       @Adi_8002 Kadrun mau demo" di jakarta, sementa...
                              ...                        
9995    @umar_hasibuan75 @KPK_RI @paijodirajo @yudihar...
9996    (24)Coba biaya Formula E yang sampai triliunan...
9997    Loh, ada apa ini?\nJakpro kan  penyelenggara F...
9998    Direktur Eksekutif LKAB Rudi S Kamri mengataka...
9999    Loh, ada apa ini?\nJakpro kan  penyelenggara F...
Name: content, Length: 9561, dtype: object

In [98]:
tweets_df = pd.DataFrame()

tweets_df['tweets'] = pd.concat([df1_indo.content, df2_indo.tweet], ignore_index=True)
print(tweets_df.shape)
tweets_df

(12521, 1)


,tweets
0,Dalam dua bulan Jakarta akan menjadi tuan ruma...
1,KPK tegaskan tak main-main usut dugaan korupsi...
2,Pengaspalan sirkuit Formula E Jakarta di kawas...
3,https://t.co/CJ0pw6Hu0O\nKlo dilaporin dalangn...
4,"@Adi_8002 Kadrun mau demo"" di jakarta, sementa..."
...,...
12516,Wakil Ketua Komisi E DPRD DKI Anggara Wicitra ...
12517,"@AlfathMauzy @f1speed_indo ""Wow ini lah tampak..."
12518,@asumsico Kirain formula e yg mau ditambahin jg
12519,@danugraha54 @YZhireng @haluanghabang @Dennysi...


In [102]:
tweets_df = tweets_df.drop_duplicates(ignore_index=True)
tweets_df

,tweets
0,Dalam dua bulan Jakarta akan menjadi tuan ruma...
1,KPK tegaskan tak main-main usut dugaan korupsi...
2,Pengaspalan sirkuit Formula E Jakarta di kawas...
3,https://t.co/CJ0pw6Hu0O\nKlo dilaporin dalangn...
4,"@Adi_8002 Kadrun mau demo"" di jakarta, sementa..."
...,...
11606,lha koq liatnya mirip sirkuit Formula E di Anc...
11607,"@AlfathMauzy @f1speed_indo ""Wow ini lah tampak..."
11608,@asumsico Kirain formula e yg mau ditambahin jg
11609,@danugraha54 @YZhireng @haluanghabang @Dennysi...


In [143]:
def clean_data(series):
    clean_tweets = []
    for tweet in series:
        tweet = re.sub("@[A-Za-z0-9]\w+","",tweet) #Remove @ sign
        tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
        tweet = emoji.replace_emoji(tweet, '') # Remove emoji
        tweet = tweet.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
        tweet = tweet.strip()
        clean_tweets.append(tweet)
    return clean_tweets

tweets_df['clean_tweets'] = clean_data(tweets_df.tweets)
tweets_df

,tweets,clean_tweets
0,Dalam dua bulan Jakarta akan menjadi tuan ruma...,Dalam dua bulan Jakarta akan menjadi tuan ruma...
1,KPK tegaskan tak main-main usut dugaan korupsi...,KPK tegaskan tak main-main usut dugaan korupsi...
2,Pengaspalan sirkuit Formula E Jakarta di kawas...,Pengaspalan sirkuit Formula E Jakarta di kawas...
3,https://t.co/CJ0pw6Hu0O\nKlo dilaporin dalangn...,"Klo dilaporin dalangnya siapa, apa brani ngus..."
4,"@Adi_8002 Kadrun mau demo"" di jakarta, sementa...","Kadrun mau demo"" di jakarta, sementara gubernu..."
...,...,...
11606,lha koq liatnya mirip sirkuit Formula E di Anc...,lha koq liatnya mirip sirkuit Formula E di Anc...
11607,"@AlfathMauzy @f1speed_indo ""Wow ini lah tampak...","""Wow ini lah tampak sirkuit formula e di Indo ..."
11608,@asumsico Kirain formula e yg mau ditambahin jg,Kirain formula e yg mau ditambahin jg
11609,@danugraha54 @YZhireng @haluanghabang @Dennysi...,"Memang ketololan cebong gk ketulungan, disamar..."


In [144]:
print(tweets_df.tweets[3], '\n')
print(tweets_df.clean_tweets[3])

https://t.co/CJ0pw6Hu0O
Klo dilaporin dalangnya siapa, apa @KPK_RI brani ngusut+nangkap? Wong kasus formula e dkk di Jakarta aja di diemin 😅😅😅, apa lg kasus ini 😅 

Klo dilaporin dalangnya siapa, apa  brani ngusut+nangkap? Wong kasus formula e dkk di Jakarta aja di diemin , apa lg kasus ini


In [152]:
from nltk.corpus import stopwords
stopwords.words('indonesian')[:10]

['ada',
 'adalah',
 'adanya',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhir']